# Nova Tarefa - Implantação

Preencha aqui com detalhes sobre a tarefa.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
# adicione imports aqui...

class Model:
    def __init__(self):
        # adicione seu código aqui...
        pass

    def predict(self, X, feature_names, meta=None):
        # adicione seu código aqui...
        return X